In [2]:
import pandas as pd
import numpy as np
import requests
import time
import re
import os
from request_simec import login, get_url, post_url
from restrictions import exemplares_aceitos, formatos_aceitos

In [41]:
df = pd.read_csv(
         'dados/TODOS.csv',
         dtype=str
    )

In [64]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
proxies = {
 'http': 'http://192.168.0.105:3128',
 'https': 'http://192.168.0.105:3128',
}
s = requests.Session()
s.get(get_url)
s.post(post_url, data=login)

<Response [200]>

In [24]:
def login ():
    s = requests.Session()
    s.proxies = proxies
    s.get(get_url, proxies=proxies)
    s.post(post_url, data=login, proxies=proxies)
    
    return s

In [25]:
s = login()

TypeError: 'function' object is not iterable

In [65]:
i = 1
for index, row in df.iterrows():  
    if (row['Extensão do Arquivo'] == 'pdf') and (row['Deve ser baixado'] == 'Sim') and (row['Baixado'] == 'Não'):
        print(str(i) + ' - ' + row['id'] + ' ' + row['Nome do Arquivo'] + ' ' + row['Extensão do Arquivo'])
        print

        try:
            r = s.get(row['Arquivo'], headers=headers)
        except Exception as e:
            print(e)
            break


        filename = 'arquivos/' + row['Edital'] + '/' + row['id'] + '/' + row['Nome do Arquivo']
        print(r.headers)

        if "Content-Lenght" in r.headers.keys():
            print('DEU RUIM')
            break

        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "wb") as f:
            f.write(r.content)

        df.at[index, 'Baixado'] = 'Sim'
        df.at[index, 'Integridade'] = 'Verificar'


        print('BAIXADO!')

        i += 1

        df.to_csv('dados/TODOS.csv', index=False)

        time.sleep(2)
        s.get(get_url)    
        #print('zzz\n')
        time.sleep(5)

!calc()
    

1 - 6346 0020P19031002DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:29:55 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0020P19031002DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=284', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
2 - 6347 COL1TITULO2_MP_2_CIENCIAS_DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:30:06 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL1TITULO2_MP_2_CIENCIAS_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=300', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'C

17 - 6441 COL2TITULO4_MP_4_HISTORIA_DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:32:57 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL2TITULO4_MP_4_HISTORIA_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=294', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
18 - 6447 0036P19041005DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:33:06 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0036P19041005DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=291', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 

33 - 6523 COL2TITULO3_MP_3_GEOGRAFIA_DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:36:46 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL2TITULO3_MP_3_GEOGRAFIA_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=289', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
34 - 6530 0042P19051004DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:37:01 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0042P19051004DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=297', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'

49 - 6637 COL1TITULO2_MP_2_CIENCIASHISTORIAGEOGRAFIA_DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:40:15 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL1TITULO2_MP_2_CIENCIASHISTORIAGEOGRAFIA_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=293', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
50 - 6641 COL1TITULO3_MP_3_CIENCIASHISTORIAGEOGRAFIA_DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:40:27 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL1TITULO3_MP_3_CIENCIASHISTORIAGEOGRAFIA_DM_FNDE.pdf', 'K

65 - 6723 COL1TITULO5_MP_5_MATEMATICA_DM_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:44:56 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL1TITULO5_MP_5_MATEMATICA_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=298', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
66 - 6727 0077P19031_1_MP_MD_FNDE.pdf pdf
{'Date': 'Tue, 06 Apr 2021 23:45:21 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0077P19031_1_MP_MD_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=287', 'Connection': 'Keep-Alive', 'Transfer-Encoding': '

...

177 - 7305 COL306TITULO6_MP_5_MATEMATICA_DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:14:22 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL306TITULO6_MP_5_MATEMATICA_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=293', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
178 - 7313 0069P19041001DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:14:32 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0069P19041001DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=287', 'Connection': 'Keep-Alive', 'Transfer-Encoding': '

193 - 7400 COL301TITULO1_MP_1_CIENCIAS_DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:17:23 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL301TITULO1_MP_1_CIENCIAS_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=269', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
194 - 7410 0088P19031002DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:17:37 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0088P19031002DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=295', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chun

209 - 7488 COL301TITULO5_MP_1_MATEMATICA_DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:21:10 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL301TITULO5_MP_1_MATEMATICA_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=299', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
210 - 7496 0092P19021002DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:21:29 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0092P19021002DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=283', 'Connection': 'Keep-Alive', 'Transfer-Encoding': '

225 - 7550 0096P19011001DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:24:42 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0096P19011001DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=287', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
226 - 7551 COL303TITULO1_MP_1_LINGUAPORTUGUESA_DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:24:52 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL303TITULO1_MP_1_LINGUAPORTUGUESA_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=296', 'Connection': 'Keep-Alive', 'Transfer-

241 - 7609 COL301TITULO6_MP_4_LINGUAPORTUGUESA_DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:27:24 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=COL301TITULO6_MP_4_LINGUAPORTUGUESA_DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=291', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}
BAIXADO!
242 - 7615 0097P19011005DM_FNDE.pdf pdf
{'Date': 'Wed, 07 Apr 2021 00:27:37 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=0097P19011005DM_FNDE.pdf', 'Keep-Alive': 'timeout=5, max=286', 'Connection': 'Keep-Alive', 'Transfer-

In [10]:
for index, row in df.iterrows():
  if (row['id'] in ['3974', '3990', '4670']):
    df.at[index, 'Baixado'] = 'Sim'
    print(row['id'] + ' ' + row['Baixado'])

3974 Sim
3990 Sim
4670 Sim


In [ ]:
df[]

In [25]:
for index, row in df.iterrows():
  if (row['Extensão do Arquivo'] == 'text/html; charset=ISO-8859-1') and (row['Deve ser baixado'] == 'Sim') and (int(row['id']) >= 6614):
    df.at[index, 'Baixado'] = 'Não'

!calc()

In [11]:
df.to_csv('dados/TODOS.csv', index = False)

In [3]:
for index, row in df.iterrows():
  if (row['Formato'] in formatos_aceitos) and (row['Exemplar'] in exemplares_aceitos):
    df.at[index, 'Deve ser baixado'] = 'Sim'
    df.at[index, 'Baixado'] = 'Não'

    header = s.head(row['Arquivo'])
    
    df.at[index, 'Header'] = header.headers
    
    
    if "content-disposition" in header.headers.keys():
        content_disposition = header.headers['content-disposition']
        
        try:
            f_name = re.findall("filename=(.+)", content_disposition)[0]
        except IndexError:
            f_name = content_disposition
            
    else:
        f_name = "NO CONTENT-DISPOSITION"
        
    if "Content-Type" in header.headers.keys():
        content_type = header.headers['Content-Type']
        
        try:
            c_type = re.findall("application/(.+)", content_type)[0]
        except IndexError:
            c_type = content_type
        
    else:
        c_type = "NO CONTENT-TYPE"
        
    
    df.at[index, 'Nome do Arquivo'] = f_name
    df.at[index, 'Extensão do Arquivo'] = c_type
    
    print(row['id'] + ' ' + f_name + ' ' + c_type)

  else:
    df.at[index, 'Deve ser baixado'] = 'Não'
    print(row['id'] + ' ****** ' + 'NÃO DEVE SER BAIXADO')
    

NameError: name 'df' is not defined